In [ ]:
# Copyright (c) 2021  IBM Corporation
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

Run this file after merging annotation with merge_json.ipyb

The goal of this file is to clean the annotations and put the the format for training, since many annotations of blocks are strangely labeled.

In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

import json
import os

import numpy as np
from shapely.geometry import Polygon

from collections import defaultdict
import cv2
import matplotlib.pyplot as plt
from skimage.draw import polygon2mask
from tqdm import tqdm
import copy
import torch

from preprocessing import extract_block_info, datadict2sample

In [ ]:
"""
SETTING PARAMETERS
"""

# json file location
json_anno_file = 'dataset/annotations_gt.json'


# processing options
dim_min_img = 512 # need to be scaled to 256 in data augmentation
to_save = True
save_path = 'dataset/processed_data'

In [ ]:
# load json
with open(json_anno_file) as json_file:
    json_obj = json.load(json_file)
    json_file.close()
    
    
# get cat_id of 'block' class
categories = json_obj['categories']
id_cat_block = [i['id'] for i in categories if i['name']=='block'][0]

# get the category relablers
cat_relabel = {}
counter_cat = 0
for i, cat_tmp in enumerate(categories):
    if cat_tmp['name'] == 'block':
        continue
        
    cat_relabel[cat_tmp['id']] = counter_cat
    counter_cat += 1
    
    
# get the list of annotation belonging to each image
annos = json_obj['annotations']

# compute the img-annotation dictionary
img_anno_dict = defaultdict(list)
for it_anno in range(len(annos)):
    img_id = annos[it_anno]['image_id']
    img_anno_dict[img_id].append(annos[it_anno])
    
# save the relabling and categorieas
dict_categories = {'cat_relabel': cat_relabel, 'categories': categories}
if not os.path.isdir(save_path):
    os.mkdir(save_path)
np.save(os.path.join(save_path, 'categories'), dict_categories)

In [ ]:
# Input feature = rgb image + additional channels with segmentation label for each object

In [ ]:
# preprocess the data
counter_valid = 0
for it_img in tqdm(range(len(json_obj['images']))):

    # get image info
    img_info = json_obj['images'][it_img]
    img_id = img_info['id']
    img_anno_ = img_anno_dict[img_id]

    # extract info
    img_anno = extract_block_info(img_anno_, id_cat_block, cat_relabel=cat_relabel)

    if len(img_anno['bbox']) <= 1:
        continue
    
    # read image
    img_loc = img_info['file_name']
    img_bgr = cv2.imread('dataset/raw/images/{}'.format(img_loc))
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    # scale image
    scale_img = dim_min_img/np.min(img_rgb.shape[:2])
    kernel_size = int(np.ceil(np.min(img_rgb.shape[:2])/256)*2+1)
    sigma = kernel_size/6
    img_tmp = cv2.GaussianBlur(img_rgb, [kernel_size, kernel_size], sigma) # blur image befor resize
    img_rgb_scaled = cv2.resize(img_tmp, (0,0), fx=scale_img, fy=scale_img)
    img_shape = img_rgb_scaled.shape

    # scale the segm
    img_anno['orig_segm'] = img_anno['segm']
    img_anno['segm'] = scale_img*img_anno['segm']

    # add the mask into the dictionary
#     img_anno['mask'] = masks
    img_anno['img_rgb_scaled'] = img_rgb_scaled
    img_anno['filename'] = img_loc
    img_anno['img_id'] = img_id
        
    counter_valid+= 1
    
    # save to disk
    if to_save:
        img_loc_path, img_ext = os.path.splitext(img_loc)
        img_folder, img_name = os.path.split(img_loc_path)


        folder_to_save_tmp = os.path.join(save_path, img_folder)
        if not os.path.isdir(folder_to_save_tmp):
            os.mkdir(folder_to_save_tmp)
        np.save(os.path.join(save_path, img_folder, img_name), img_anno)

print("Valid data with >1 bboxes: {}".format(counter_valid))

# Done
The rest of this section is for visualization

In [ ]:
# load categories
dict_cat = np.load('dataset/processed_data/categories.npy', allow_pickle=True).item()
categories = dict_cat['categories']
cat_relabel = dict_cat['cat_relabel']

#load processed data and convert to Sample
datadict = np.load('dataset/processed_data/20210312_1_Station/IMG_0620.npy', allow_pickle=True).item()
n_categories = len(cat_relabel)

In [ ]:
import visualization as vis

fig, axs = plt.subplots(1, 1, figsize=(15, 15))
# vis.visualize_groups(axs, np.repeat(im_bw[:,:,None], 3, axis=2) , bboxes, output_block_labels, None)
vis.visualize_groups(axs, datadict['img_rgb_scaled'], datadict['segm'], datadict['node_group_id'], None)